In [1]:

from __future__ import division
from __future__ import print_function
from __future__ import absolute_import
from distutils.version import StrictVersion
import pytesseract
import shutil
import tensorflow as tf
import re
import pandas as pd
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'    # Suppress TensorFlow logging (1)
import inflect
import uuid
from shapely.geometry import Polygon
from mining_pages_utils.image_ocr_utils import double_to_singlepage, load_page, cut_image, ocr_pre_processing, ocr_post_processing_pageid, cut_image_savetemp, cut_image_figid, ocr_post_processing_figid
from mining_pages_utils.dataframe_utils import get_page_labelmap_as_df, get_figid_labelmap_as_df, extract_page_detections, extract_page_detections_new,unfold_pagedetections, page_detections_toframe, extract_detections_figureidv2,humanreadID
from mining_pages_utils.dataframe_utils import extract_pdfid, filter_best_page_detections, select_pdfpages, choose_pageid, filter_best_vesselprofile_detections, merge_info,  provide_pagelist, provide_pdf_path, get_pubs_and_configs, pdf_to_imagev2, handleduplicate_humanreadID
from mining_pages_utils.json_utils import create_find_JSONL, create_constructivisttype_JSONL, create_normativtype_JSONL, create_drawing_JSONL, create_catalog_JSONL, create_trench_JSONL
from mining_pages_utils.tensorflow_utils import create_tf_example_new, create_tf_figid, run_inference_for_page_series, run_inference_for_figure_series, build_detectfn, Df2TFrecord, split
from mining_pages_utils.request_utils import getZenonInfo

if StrictVersion(tf.version.VERSION) < StrictVersion('2.3.0'):
    raise ImportError(
        'Please upgrade your TensorFlow installation to v1.9.* or later!')

tf.get_logger().setLevel('ERROR')           # Suppress TensorFlow logging (2)

# Enable GPU dynamic memory allocation
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

INPUTDIRECTORY = '/home/images/apply' 
GRAPH = '/frozen_inference_graph.pb'
LABELS = '/label_map.pbtxt'
PAGE_MODEL = '/home/models/faster_rcnn_resnet101_v1_1024x1024_coco17_OCKquick_miningpagesv9_posttrain'
SAVEDMODEL = '/saved_model'
FIGID_MODEL = '/home/models/faster_rcnn_resnet101_v1_1024x1024_coco17_miningfiguresv3'
SEG_MODEL = '/home/models/shape_segmentation/train_colab_20200610.h5'
OUTPATH = '/home/images/OUTPUT/'
VESSELLPATH = OUTPATH + 'vesselprofiles/'
SEGMENTPATH = OUTPATH + 'segmented_profiles/'
CSVOUT = OUTPATH + 'mining_pages_allinfo.csv'
CLEANCSVOUT = OUTPATH + 'mining_pages_clean.csv'









Segmentation Models: using `tf.keras` framework.


In [4]:
publist = get_pubs_and_configs(INPUTDIRECTORY)
publist = publist.apply(getZenonInfo, axis=1)
pdflist = provide_pdf_path(publist)
pdflistv2 = pdflist.apply(pdf_to_imagev2, axis=1)
pagelist = provide_pagelist(pdflistv2)
pagelist = pagelist.apply(extract_pdfid, axis=1)
pagelist = double_to_singlepage(pagelist)
pagelist = provide_pagelist(pdflist)
pagelist = pagelist.apply(extract_pdfid, axis=1)
pagelist = select_pdfpages(pagelist)





For pub_key: ZenonID found pub_info!


NameError: name 'pdf_to_imagev2' is not defined

In [3]:

for path in [VESSELLPATH, SEGMENTPATH]:
    if not os.path.exists(path):
        os.makedirs(path)







all_detections_step1 = pd.DataFrame()
miningpagesdetectfn = build_detectfn(PAGE_MODEL + SAVEDMODEL)
for index, row in pagelist.iterrows():
    print('Page ' + os.path.basename(row['page_path']))

    row, page_imgnp = load_page(row)
    input_tensor = tf.convert_to_tensor(page_imgnp)
    input_tensor = input_tensor[tf.newaxis, ...]
    detections = miningpagesdetectfn(input_tensor)
    del page_imgnp
    del input_tensor
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                   for key, value in detections.items()}
    detections['num_detections'] = num_detections
    row['page_detections']= detections
    all_detections_step1 = all_detections_step1.append(row)










Loading model...Done! Took 11.802120685577393 seconds
Page ock 1.pdf0001-4.png
/home/images/apply/ZenonID_000066595/ock 1.pdf0001-4.png
Page ock 1.pdf0001-5.png
/home/images/apply/ZenonID_000066595/ock 1.pdf0001-5.png
Page ock 1.pdf0001-1.png
/home/images/apply/ZenonID_000066595/ock 1.pdf0001-1.png
Page ock 1.pdf0001-3.png
/home/images/apply/ZenonID_000066595/ock 1.pdf0001-3.png
Page ock 1.pdf0001-2.png
/home/images/apply/ZenonID_000066595/ock 1.pdf0001-2.png


In [11]:
#all_detections_step2, keylist =unfold_pagedetections(all_detections_step1)
#all_detections_step22 = extract_page_detections(all_detections_step2, keylist, category_index=get_page_labelmap_as_df(PAGE_MODEL + LABELS))
all_detections_step2 = page_detections_toframe(all_detections_step1).drop(columns='page_detections')

page_category_index = get_page_labelmap_as_df(PAGE_MODEL + LABELS)
all_detections_step2 = all_detections_step2.merge(page_category_index, on=['detection_classes'], how='left')

In [12]:
pageids = filter_best_page_detections(all_detections_step2 , lowest_score=0.8)
bestpages = choose_pageid(pageids)
pageid_raw = pd.DataFrame()

#perform ocr page number
for index, row in bestpages.iterrows():
    img = cut_image(row)
    img2 = ocr_pre_processing(img)
    result = pytesseract.image_to_string(img2, config=row['pageid_config'])
    row['newinfo'] = result
    
    pageid_raw = pageid_raw.append(row)
all_detections_step3 = merge_info(all_detections_step2, pageid_raw)
all_detections_step3 = all_detections_step3.apply(ocr_post_processing_pageid, axis=1)

figures = filter_best_vesselprofile_detections(all_detections_step3, lowest_score= 0.7)

['pageid']


In [6]:
for index, row in figures.iterrows():
    print(row['detection_boxes'])

[0.63001263 0.48634285 0.77959555 0.9058297 ]
[0.4751732  0.49714524 0.62916595 0.897826  ]
[0.33474877 0.4870547  0.47320363 0.89732325]
[0.44515532 0.04732963 0.67301303 0.4452881 ]
[0.4040281  0.5538879  0.4344727  0.84998566]
[0.27029854 0.04556972 0.4398847  0.45403546]
[0.5439941  0.5566677  0.57744384 0.84295213]
[0.40285066 0.10167895 0.43539044 0.4125495 ]
[0.12388347 0.49677294 0.33228314 0.9032466 ]
[0.24401225 0.5528629  0.28130013 0.8397409 ]
[0.5422908  0.10121678 0.5654415  0.39487875]
[0.67570287 0.03774619 0.7758362  0.4726938 ]
[0.43653873 0.5452385  0.47069663 0.8453417 ]
[0.17457859 0.04920328 0.2716194  0.44978723]
[0.58068305 0.54428923 0.619059   0.84790075]
[0.5675623  0.11187372 0.590955   0.397252  ]
[0.0824903  0.04793471 0.17074268 0.44576997]
[0.5126598  0.10499637 0.5417769  0.400975  ]
[0.24799062 0.10751471 0.26732564 0.40135685]
[0.2841725  0.553341   0.30629066 0.84152466]
[0.19452667 0.5512     0.23953223 0.8455927 ]
[0.1472892  0.10698174 0.16706876 

In [7]:
#detect figure id


figures_step1 = pd.DataFrame()
miningfiguresdetectfn = build_detectfn(FIGID_MODEL + SAVEDMODEL)
for index, row in figures.iterrows():
    print('Figure from ' + os.path.basename(row['page_path']))
    row, figure_imgnp = cut_image_savetemp(row, VESSELLPATH)
    input_tensor = tf.convert_to_tensor(figure_imgnp)
    input_tensor = input_tensor[tf.newaxis, ...]
    detections = miningfiguresdetectfn(input_tensor)
    del figure_imgnp
    del input_tensor
    num_detections = int(detections.pop('num_detections'))
    
    detections = {'figid_' + key: value[0, :num_detections].numpy()
                   for key, value in detections.items()}
    row['figid_detections'] = detections
    detections['figid_num_detections'] = num_detections
    row = extract_detections_figureidv2(row)         
    figures_step1 = figures_step1.append(row)

figid_category_index = get_figid_labelmap_as_df(PAGE_MODEL + LABELS)
figures_step2 = figures_step1.merge(
figid_category_index, on=['figid_detection_classes'], how='left')






Loading model...Done! Took 11.48296070098877 seconds
Figure from ock 1.pdf0001-4.png
Figure from ock 1.pdf0001-4.png
Figure from ock 1.pdf0001-4.png
Figure from ock 1.pdf0001-4.png
Figure from ock 1.pdf0001-4.png
Figure from ock 1.pdf0001-4.png
Figure from ock 1.pdf0001-4.png
Figure from ock 1.pdf0001-4.png
Figure from ock 1.pdf0001-4.png
Figure from ock 1.pdf0001-4.png
Figure from ock 1.pdf0001-4.png
Figure from ock 1.pdf0001-4.png
Figure from ock 1.pdf0001-4.png
Figure from ock 1.pdf0001-4.png
Figure from ock 1.pdf0001-4.png
Figure from ock 1.pdf0001-4.png
Figure from ock 1.pdf0001-4.png
Figure from ock 1.pdf0001-4.png
Figure from ock 1.pdf0001-4.png
Figure from ock 1.pdf0001-4.png
Figure from ock 1.pdf0001-4.png
Figure from ock 1.pdf0001-4.png
Figure from ock 1.pdf0001-4.png
Figure from ock 1.pdf0001-4.png
Figure from ock 1.pdf0001-4.png
Figure from ock 1.pdf0001-4.png
Figure from ock 1.pdf0001-4.png
Figure from ock 1.pdf0001-4.png
Figure from ock 1.pdf0001-4.png
Figure from ock 1.p

In [8]:
#perform ocr figid
figures_step3 = pd.DataFrame()
for index, row in figures_step2.iterrows():
    print('OCR ' + os.path.basename(row['figure_path']))
    img = cut_image_figid(row)
    img2 = ocr_pre_processing(img)
    row['figid_raw'] = pytesseract.image_to_string(img2, config=row['pageid_config'])
    del img
    del img2
    figures_step3 = figures_step3.append(row)
figures_step3 = figures_step3.apply(ocr_post_processing_figid, axis=1)
figures_step3 = figures_step3.apply(humanreadID, axis=1)
figures_step3 = handleduplicate_humanreadID(figures_step3)

OCR 58338148-3975-4d71-90f7-4c40967703b4.png
OCR ff361d71-4fb9-4f20-838a-62ae2c240b4e.png
OCR 4910732e-8ce3-49c0-af5d-349f733e50e1.png
OCR bdee2edd-b67d-4474-8009-c2822faf29c4.png
OCR bf73f5a1-8c7e-4f9a-9bbb-4091858e8409.png
OCR 24dce7e4-1507-4c25-b942-72e8f52ba241.png
OCR 0686d569-49b6-40d8-a1ae-2102a54bcb56.png
OCR 61fea08c-05d4-46e1-b79a-6b3075b3a499.png
OCR f72f3f9d-123e-4469-a7ed-5c0ceb951de3.png
OCR 4b560762-c60c-480c-8128-3d673992622d.png
OCR d83b066e-6381-41c1-b911-6af9d90397bc.png
OCR 9a94c7c8-2ec4-470c-94e0-ea6a2fa5b89c.png
OCR 36c37f11-7981-4287-a676-d7b84662ed39.png
OCR 38db68a4-b13d-4505-942d-3d203f5cd3c8.png
OCR b249cb62-d6e5-4820-911c-4bf3618c1a17.png
OCR b638bff0-0505-4332-83e5-4bac30fe2db9.png
OCR d4bdf09a-d176-445d-bdef-0d4aee4f2176.png
OCR b2f3e581-a0a5-45b5-8e07-7334888c4666.png
OCR b3e3467b-c1af-4821-a55f-ede3c6cc2431.png
OCR b2e5534a-7ee0-43e3-a898-e7d595693bb7.png
OCR 759f546f-366b-486e-90e9-1455d04fc0cd.png
OCR f9157966-03a2-4679-b187-99e19b80d8b4.png
OCR 74d409

In [9]:

with open(OUTPATH + 'catalogs.jsonl', 'w') as f:
    pubs = figures_step3[['pub_key', 'pub_value']].drop_duplicates()
    pubs.apply(create_catalog_JSONL, file=f, axis=1)
with open(OUTPATH + 'trenches.jsonl', 'w') as f:
    pubs = figures_step3[['pub_key', 'pub_value']].drop_duplicates()
    pubs.apply(create_trench_JSONL, file=f, axis=1)
with open(OUTPATH + 'types.jsonl', 'w') as f:
    figures_step3.apply(create_constructivisttype_JSONL, file=f, axis=1)
with open(OUTPATH + 'types_standalone.jsonl', 'w') as f:
    figures_step3.apply(create_normativtype_JSONL, file=f, axis=1)
with open(OUTPATH + 'finds.jsonl', 'w') as f:
    figures_step3.apply(create_find_JSONL, file=f, axis=1)
with open(OUTPATH + 'drawings.jsonl', 'w') as f:
    figures_step3.apply(create_drawing_JSONL, file=f, axis=1)







figures_step3.to_csv(CSVOUT)
figures_clean = figures_step3[['pub_key','pub_value','figure_tmpid','HRID','detection_scores', 'detection_classesname','page_imgname','pageid_raw','figid_raw','pageid_clean','figid_clean','pageinfo_raw','figure_path','page_path']]
figures_clean.to_csv(CLEANCSVOUT)


#Profile segmentation
#print('Perform image segmentation')
#run_vesselprofile_segmentation(VESSELLPATH, SEGMENTPATH, SEG_MODEL)

In [10]:
shutil.copyfile(PAGE_MODEL + LABELS, OUTPATH +'pages_label_map.pbtxt')

mining_pages_detections = figures_step3.append(bestpages)
mining_pages_detections2 = mining_pages_detections.reindex(axis=0)
pages_grouped = mining_pages_detections2.groupby('pub_name')

for name, group in pages_grouped:
    imgdir = os.path.join(OUTPATH, name + '_pages/')
    if not os.path.exists(imgdir):
        os.makedirs(imgdir)
    Df2TFrecord(group, 'page_path', OUTPATH + name +'_pages.tfrecord')
    for index, row in group.iterrows():
        imgoutpath = imgdir + os.path.basename(row['page_path'])
        print(imgoutpath)
        if not os.path.exists(imgoutpath):
            shutil.copyfile(row['page_path'], imgoutpath)

shutil.copyfile(FIGID_MODEL + LABELS, OUTPATH + 'figures_label_map.pbtxt')
figures_step3 = figures_step3[figures_step3.detection_boxes.notnull()]

figures_grouped = figures_step3.groupby('pub_name')

for name, group in figures_grouped:
    imgdir = os.path.join(OUTPATH, name + '_figures/')
    if not os.path.exists(imgdir):
        os.makedirs(imgdir)
    Df2TFrecord(group, 'figure_path', OUTPATH + name +'_figures.tfrecord')
    for index, row in group.iterrows():
        imgoutpath = imgdir + os.path.basename(str(row['figure_path']))
        print(imgoutpath)
        if not os.path.exists(imgoutpath):
            shutil.copyfile(row['figure_path'], imgoutpath)


[data(filename='/home/images/apply/ZenonID_000066595/ock 1.pdf0001-1.png', object=              classlist                               detection_boxes  \
56   [pageid, pageinfo]  [0.24463052, 0.311545, 0.5171886, 0.9035255]   
600  [pageid, pageinfo]  [0.7880302, 0.25431642, 0.8513363, 0.948524]   

     detection_classes detection_classesname  detection_scores  \
56                 3.0              stampbox          0.868527   
600                1.0                pageid          0.955734   

     expectDoublepages                          figid_detection_boxes  \
56                 1.0  [0.835294, 0.21058702, 0.9645727, 0.51672506]   
600                1.0                                            NaN   

     figid_detection_classes figid_detection_classesname  \
56                       1.0                      pageid   
600                      NaN                         NaN   

     figid_detection_scores  ...              patternHRID  pub_key  pub_name  \
56                